# Чтение данных

In [1]:
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd

train = pd.read_csv("data/train.tsv.zip")
test = pd.read_csv("data/test.tsv")
sample_submission = pd.read_csv("data/sample_submission.tsv")

In [3]:
train['is_train']=1
test['is_train']=0
full_data = pd.concat([train.reset_index(),test.reset_index()], axis=0, sort=False).drop(['index'], axis=1)
full_data.reset_index(inplace=True)
full_data.drop('index', axis=1, inplace=True)
full_data.shape

(74473, 67)

# Leakage

In [4]:
leakeage_data = full_data[['item_id','year', 'week', 'shift', 'y', 'Num']]

In [5]:
#нумеруме строки в пределах одного айтема
g=leakeage_data.groupby('item_id')
leakeage_data['rn'] = g['Num'].rank(method='min')

In [6]:
#нумерацию строк разбиваем на тройки
leakeage_data['rn_grouped'] = leakeage_data['rn']%3
leakeage_data.loc[leakeage_data['rn_grouped']==0,'rn_grouped']=3
#нумеруем тройки
leakeage_data['rn_group'] = leakeage_data['rn']//3

#определяем значение предыдущей тройки
leakeage_data['rn_group_prev'] = leakeage_data['rn_group']-1
#определяем значение предыдущей строки
leakeage_data.loc[leakeage_data['rn_grouped']==1, 'rn_grouped_prev']=2
leakeage_data.loc[leakeage_data['rn_grouped']==2, 'rn_grouped_prev']=3

In [7]:
leakeage_data[leakeage_data['item_id']==20438733][-15:]

,item_id,year,week,shift,y,Num,rn,rn_grouped,rn_group,rn_group_prev,rn_grouped_prev
71334,20438733,2014,53,1,7230.0,347052,316.0,1.0,105.0,104.0,2.0
71558,20438733,2015,1,2,829.0,347276,317.0,2.0,105.0,104.0,3.0
71783,20438733,2015,2,3,4181.0,347501,318.0,3.0,106.0,105.0,NaN
72005,20438733,2015,1,1,829.0,348395,319.0,1.0,106.0,105.0,2.0
72230,20438733,2015,2,2,4181.0,348620,320.0,2.0,106.0,105.0,3.0
72455,20438733,2015,2,1,4181.0,349517,322.0,1.0,107.0,106.0,2.0
72679,20438733,2015,3,3,NaN,348844,321.0,3.0,107.0,106.0,NaN
72905,20438733,2015,3,2,NaN,349741,323.0,2.0,107.0,106.0,3.0
73134,20438733,2015,4,3,NaN,349970,324.0,3.0,108.0,107.0,NaN
73357,20438733,2015,3,1,NaN,350413,325.0,1.0,108.0,107.0,2.0


In [8]:
leakeage_data_merged = pd.merge(leakeage_data, leakeage_data, suffixes=['', '_pdata'],
         right_on=['item_id','rn_group', 'rn_grouped'], 
         left_on=['item_id', 'rn_group_prev', 'rn_grouped_prev'], how='left' 
        )
leakeage_data_merged.reset_index(inplace=True)
leakeage_data_merged.drop('index', axis=1, inplace=True)
leakeage_data_merged.shape

(74473, 21)

In [9]:
leakeage_data[leakeage_data['item_id']==20438733][-20:]

,item_id,year,week,shift,y,Num,rn,rn_grouped,rn_group,rn_group_prev,rn_grouped_prev
70216,20438733,2014,52,2,10894.0,343918,311.0,2.0,103.0,102.0,3.0
70439,20438733,2014,53,3,7230.0,344141,312.0,3.0,104.0,103.0,NaN
70665,20438733,2014,52,1,10894.0,345486,313.0,1.0,104.0,103.0,2.0
70888,20438733,2014,53,2,7230.0,345709,314.0,2.0,104.0,103.0,3.0
71112,20438733,2015,1,3,829.0,345933,315.0,3.0,105.0,104.0,NaN
71334,20438733,2014,53,1,7230.0,347052,316.0,1.0,105.0,104.0,2.0
71558,20438733,2015,1,2,829.0,347276,317.0,2.0,105.0,104.0,3.0
71783,20438733,2015,2,3,4181.0,347501,318.0,3.0,106.0,105.0,NaN
72005,20438733,2015,1,1,829.0,348395,319.0,1.0,106.0,105.0,2.0
72230,20438733,2015,2,2,4181.0,348620,320.0,2.0,106.0,105.0,3.0


In [10]:
leakeage_data_merged.columns

Index(['item_id', 'year', 'week', 'shift', 'y', 'Num', 'rn', 'rn_grouped',
       'rn_group', 'rn_group_prev', 'rn_grouped_prev', 'year_pdata',
       'week_pdata', 'shift_pdata', 'y_pdata', 'Num_pdata', 'rn_pdata',
       'rn_grouped_pdata', 'rn_group_pdata', 'rn_group_prev_pdata',
       'rn_grouped_prev_pdata'],
      dtype='object')

In [11]:
leakeage_data_merged[leakeage_data_merged['item_id']==20438733][['rn_group','rn_grouped',
                                                                 'rn_group_pdata','rn_grouped_pdata',
                                                                 'year', 'week',
                                                                 'y', 'y_pdata']][-20:]

,rn_group,rn_grouped,rn_group_pdata,rn_grouped_pdata,year,week,y,y_pdata
70216,103.0,2.0,102.0,3.0,2014,52,10894.0,6048.0
70439,104.0,3.0,NaN,NaN,2014,53,7230.0,NaN
70665,104.0,1.0,103.0,2.0,2014,52,10894.0,10894.0
70888,104.0,2.0,103.0,3.0,2014,53,7230.0,10894.0
71112,105.0,3.0,NaN,NaN,2015,1,829.0,NaN
71334,105.0,1.0,104.0,2.0,2014,53,7230.0,7230.0
71558,105.0,2.0,104.0,3.0,2015,1,829.0,7230.0
71783,106.0,3.0,NaN,NaN,2015,2,4181.0,NaN
72005,106.0,1.0,105.0,2.0,2015,1,829.0,829.0
72230,106.0,2.0,105.0,3.0,2015,2,4181.0,829.0


In [13]:
full_data['y_leaked'] = leakeage_data_merged['y_pdata']

In [15]:
full_data.to_excel('full_data_leaked.xlsx')

In [51]:
a = full_data.loc[full_data['is_train']==0,['year', 'week','item_id', 'y_leaked']]
leaked = a.drop_duplicates().dropna()

In [55]:
leaked.shape

(229, 4)

In [58]:
test_leaked = pd.merge(left=test[['Num', 'year', 'week', 'item_id']], right=leaked, how='left')

In [60]:
test_leaked.count()

Num         2016
year        2016
week        2016
item_id     2016
y_leaked     687
dtype: int64

In [61]:
test_leaked.to_excel('predictions/test_leaked.xlsx')

In [63]:
test_leaked.to_csv('predictions/test_leaked.csv', index=False)